# Contradiction Detection with MLflow Faithfulness Metrics

This notebook demonstrates how to use MLflow faithfulness metrics to detect **contradictions** in LLM outputs. A contradiction occurs when the model generates content that directly opposes or negates information stated in the provided context.

## What is Contradiction?

A contradiction is when an LLM generates:
- **Direct negation** of facts in the context (e.g., context says "X is true", answer says "X is false")
- **Opposite values** for numeric data (e.g., "increased" vs "decreased")
- **Conflicting attributes** (e.g., "red" vs "blue", "large" vs "small")
- **Temporal inconsistencies** (e.g., "before" vs "after", wrong dates)
- **Logical impossibilities** given the context

### Why Contradiction Detection Matters:
| Impact | Example |
|--------|---------|
| **Misinformation** | Spreading false information that contradicts sources |
| **Trust Erosion** | Users lose confidence when AI contradicts its sources |
| **Critical Errors** | Medical/legal/financial contradictions can cause harm |
| **Confusion** | Users don't know what to believe |

### Faithfulness Score for Contradiction:
| Score Range | Interpretation | Risk Level |
|-------------|----------------|------------|
| >= 0.70 | Consistent Response | Safe |
| 0.45 - 0.70 | Minor Inconsistency | Low |
| 0.25 - 0.45 | Likely Contradiction | Medium |
| < 0.25 | Clear Contradiction | High |


## 1. Setup and Installation


In [ ]:
%pip install -q mlflow sentence-transformers pandas numpy scikit-learn


In [ ]:
%restart_python


In [ ]:
import numpy as np
import pandas as pd
import re
from typing import List, Dict
import warnings
warnings.filterwarnings('ignore')

from sentence_transformers import SentenceTransformer, CrossEncoder
from sklearn.metrics.pairwise import cosine_similarity

print("✅ Libraries loaded successfully!")


## 2. Contradiction Detector

We use NLI (Natural Language Inference) as the primary signal for contradiction detection, since NLI models are specifically trained to identify when one text contradicts another. The model outputs three classes: entailment, neutral, and **contradiction**.


In [ ]:
class ContradictionDetector:
    """
    Detects contradictions where the generated answer directly opposes
    or negates information in the provided context.
    
    Uses NLI model's contradiction score as the primary signal.
    """

    def __init__(self, 
                 embedding_model: str = "all-MiniLM-L6-v2",
                 nli_model: str = "cross-encoder/nli-deberta-v3-small"):
        """Initialize with embedding and NLI models."""
        print(f"Loading embedding model: {embedding_model}...")
        self.embedding_model = SentenceTransformer(embedding_model)
        
        print(f"Loading NLI model: {nli_model}...")
        self.nli_model = CrossEncoder(nli_model)
        
        print("✅ Contradiction detector ready!")

    def compute_semantic_similarity(self, answer: str, context: str) -> float:
        """Compute semantic similarity between answer and context."""
        answer_embedding = self.embedding_model.encode([answer])[0]
        context_embedding = self.embedding_model.encode([context])[0]
        similarity = cosine_similarity([answer_embedding], [context_embedding])[0][0]
        return max(0, min(1, (similarity + 1) / 2))

    def compute_nli_scores(self, answer: str, context: str) -> Dict[str, float]:
        """
        Compute NLI scores: contradiction, neutral, entailment.
        Returns all three scores for analysis.
        """
        scores = self.nli_model.predict([(context, answer)])[0]
        
        if isinstance(scores, (int, float)):
            # Single score model - use sigmoid
            entailment = 1 / (1 + np.exp(-scores))
            return {
                "contradiction": 1 - entailment,
                "neutral": 0.0,
                "entailment": entailment
            }
        
        # Multi-class model: [contradiction, neutral, entailment]
        return {
            "contradiction": float(scores[0]),
            "neutral": float(scores[1]),
            "entailment": float(scores[2])
        }

    def compute_token_overlap(self, answer: str, context: str) -> float:
        """Compute token overlap between answer and context."""
        stop_words = {'the', 'a', 'an', 'is', 'are', 'was', 'were', 'be', 'been', 
                      'to', 'of', 'in', 'for', 'on', 'with', 'at', 'by', 'from',
                      'and', 'but', 'or', 'it', 'its', 'this', 'that', 'who', 'not'}
        
        answer_tokens = set(answer.lower().split()) - stop_words
        context_tokens = set(context.lower().split()) - stop_words
        
        if not answer_tokens:
            return 1.0
        
        overlap = answer_tokens.intersection(context_tokens)
        return len(overlap) / len(answer_tokens)

    def detect_contradiction(self, answer: str, context: str) -> Dict:
        """
        Detect if an answer contradicts the context.
        
        Returns:
            Dictionary with scores and contradiction verdict
        """
        semantic = self.compute_semantic_similarity(answer, context)
        nli_scores = self.compute_nli_scores(answer, context)
        overlap = self.compute_token_overlap(answer, context)
        
        # For contradiction detection, we focus on the contradiction score
        # High contradiction score = bad (contradicts context)
        # High entailment score = good (supported by context)
        
        # Faithfulness score: high entailment + low contradiction = good
        # We invert the scoring: higher score = less contradiction
        faithfulness = nli_scores["entailment"] - (0.5 * nli_scores["contradiction"])
        faithfulness = max(0, min(1, (faithfulness + 0.5)))  # Normalize to 0-1
        
        # Combined score weighted toward NLI
        combined = 0.60 * faithfulness + 0.25 * semantic + 0.15 * overlap
        
        # Determine contradiction category based on contradiction score
        contradiction_score = nli_scores["contradiction"]
        
        if contradiction_score >= 0.7:
            category = "🚫 Clear Contradiction"
            is_contradiction = True
            risk = "High"
        elif contradiction_score >= 0.4:
            category = "❌ Likely Contradiction"
            is_contradiction = True
            risk = "Medium"
        elif contradiction_score >= 0.2:
            category = "⚠️ Minor Inconsistency"
            is_contradiction = True
            risk = "Low"
        else:
            category = "✅ Consistent Response"
            is_contradiction = False
            risk = "Low"
        
        return {
            "semantic_score": semantic,
            "nli_entailment": nli_scores["entailment"],
            "nli_neutral": nli_scores["neutral"],
            "nli_contradiction": nli_scores["contradiction"],
            "overlap_score": overlap,
            "faithfulness_score": faithfulness,
            "combined_score": combined,
            "category": category,
            "is_contradiction": is_contradiction,
            "risk_level": risk
        }

# Initialize the detector
detector = ContradictionDetector()


## 3. Test Examples: Detecting Contradictions

We'll test several types of contradictions:
1. **Numeric Contradictions** - Wrong numbers, percentages, dates
2. **Logical Contradictions** - Direct negation of facts
3. **Attribute Contradictions** - Wrong properties or characteristics
4. **Temporal Contradictions** - Wrong sequence or timing
5. **Directional Contradictions** - Opposite directions (increase/decrease, up/down)


In [ ]:
# Define test examples with context, consistent answer, and contradictory answer
test_examples = [
    {
        "name": "Example 1: Numeric Contradiction",
        "category": "Numeric",
        "context": "The company reported revenue of $50 million in Q3 2024, representing a 25% increase from the previous year.",
        "consistent_answer": "The company's Q3 2024 revenue was $50 million, up 25% year-over-year.",
        "contradictory_answer": "The company's Q3 2024 revenue was $30 million, representing a 10% decrease from last year."
    },
    {
        "name": "Example 2: Logical Contradiction",
        "category": "Logical",
        "context": "The study concluded that the new drug was effective in treating the condition, with 85% of patients showing improvement.",
        "consistent_answer": "The drug proved effective, with 85% of patients improving in the study.",
        "contradictory_answer": "The study found that the drug was ineffective, with most patients showing no improvement."
    },
    {
        "name": "Example 3: Attribute Contradiction",
        "category": "Attribute",
        "context": "The electric vehicle has a range of 300 miles on a single charge and can accelerate from 0 to 60 mph in 4 seconds.",
        "consistent_answer": "The EV offers 300 miles of range and reaches 60 mph in just 4 seconds.",
        "contradictory_answer": "The EV has a limited range of only 150 miles and slow acceleration, taking 12 seconds to reach 60 mph."
    },
    {
        "name": "Example 4: Temporal Contradiction",
        "category": "Temporal",
        "context": "World War II ended in 1945 after Germany surrendered in May and Japan surrendered in September.",
        "consistent_answer": "WWII concluded in 1945 with Germany's surrender in May and Japan's in September.",
        "contradictory_answer": "World War II ended in 1942, with Japan surrendering before Germany."
    },
    {
        "name": "Example 5: Directional Contradiction",
        "category": "Directional",
        "context": "Global temperatures have risen by 1.1°C since pre-industrial times, primarily due to greenhouse gas emissions.",
        "consistent_answer": "Global temperatures have increased by 1.1°C from pre-industrial levels due to greenhouse gases.",
        "contradictory_answer": "Global temperatures have decreased significantly since pre-industrial times, despite increased emissions."
    },
    {
        "name": "Example 6: Outcome Contradiction",
        "category": "Outcome",
        "context": "The patient's surgery was successful, and they were discharged from the hospital after 3 days of recovery.",
        "consistent_answer": "The surgery went well, and the patient left the hospital after a 3-day recovery.",
        "contradictory_answer": "The surgery failed, and the patient remained in critical condition for several weeks."
    },
    {
        "name": "Example 7: Identity Contradiction",
        "category": "Identity",
        "context": "Python is a high-level, interpreted programming language created by Guido van Rossum in 1991.",
        "consistent_answer": "Python, created by Guido van Rossum in 1991, is an interpreted high-level language.",
        "contradictory_answer": "Python is a low-level, compiled language developed by Bill Gates in the 1970s."
    }
]

print(f"📋 Loaded {len(test_examples)} contradiction test examples across {len(set(e['category'] for e in test_examples))} categories")


### 3.1 Running Contradiction Detection


In [ ]:
def print_detection_result(name: str, category: str, context: str, answer: str, result: Dict, answer_type: str):
    """Pretty print the contradiction detection result."""
    print(f"\n{'='*80}")
    print(f"📝 {name}")
    print(f"   Category: {category} | Type: {answer_type}")
    print(f"{'='*80}")
    print(f"\n📄 Context: {context[:100]}...")
    print(f"\n💬 Answer: {answer}")
    print(f"\n📊 NLI SCORES:")
    print(f"   • Entailment:     {result['nli_entailment']:.3f} (supports context)")
    print(f"   • Neutral:        {result['nli_neutral']:.3f} (unrelated)")
    print(f"   • Contradiction:  {result['nli_contradiction']:.3f} ← KEY METRIC")
    print(f"\n📊 OTHER SCORES:")
    print(f"   • Semantic:       {result['semantic_score']:.3f}")
    print(f"   • Overlap:        {result['overlap_score']:.3f}")
    print(f"   • Combined:       {result['combined_score']:.3f}")
    print(f"\n🎯 VERDICT: {result['category']}")
    print(f"   • Is Contradiction: {'YES ❌' if result['is_contradiction'] else 'NO ✓'}")
    print(f"   • Risk Level: {result['risk_level']}")

# Run detection on all examples
print("🔍 CONTRADICTION DETECTION RESULTS")
print("="*80)

all_results = []

for example in test_examples:
    # Test consistent answer
    consistent_result = detector.detect_contradiction(
        example["consistent_answer"], 
        example["context"]
    )
    print_detection_result(
        example["name"], 
        example["category"],
        example["context"], 
        example["consistent_answer"], 
        consistent_result, 
        "CONSISTENT ANSWER"
    )
    all_results.append({
        "example": example["name"],
        "category": example["category"],
        "type": "Consistent",
        "contradiction_score": consistent_result["nli_contradiction"],
        "combined_score": consistent_result["combined_score"],
        "is_contradiction": consistent_result["is_contradiction"],
        "verdict": consistent_result["category"]
    })
    
    # Test contradictory answer
    contradictory_result = detector.detect_contradiction(
        example["contradictory_answer"], 
        example["context"]
    )
    print_detection_result(
        example["name"], 
        example["category"],
        example["context"], 
        example["contradictory_answer"], 
        contradictory_result, 
        "CONTRADICTORY ANSWER"
    )
    all_results.append({
        "example": example["name"],
        "category": example["category"],
        "type": "Contradictory",
        "contradiction_score": contradictory_result["nli_contradiction"],
        "combined_score": contradictory_result["combined_score"],
        "is_contradiction": contradictory_result["is_contradiction"],
        "verdict": contradictory_result["category"]
    })


### 3.2 Summary Results Table


In [ ]:
# Create summary DataFrame
results_df = pd.DataFrame(all_results)

print("\n" + "="*80)
print("📊 SUMMARY: CONTRADICTION DETECTION RESULTS")
print("="*80)

# Format the dataframe for display
display_df = results_df.copy()
display_df["contradiction_score"] = display_df["contradiction_score"].apply(lambda x: f"{x:.3f}")
display_df["combined_score"] = display_df["combined_score"].apply(lambda x: f"{x:.3f}")
display_df["is_contradiction"] = display_df["is_contradiction"].apply(lambda x: "❌ YES" if x else "✓ NO")
display_df.columns = ["Example", "Category", "Answer Type", "Contradiction", "Combined", "Detected?", "Verdict"]

print("\n")
print(display_df.to_string(index=False))

# Calculate accuracy
consistent_correct = sum(1 for r in all_results if r["type"] == "Consistent" and not r["is_contradiction"])
contradictory_detected = sum(1 for r in all_results if r["type"] == "Contradictory" and r["is_contradiction"])
total = len(test_examples)

print(f"\n{'='*80}")
print("📈 DETECTION ACCURACY")
print("="*80)
print(f"   • Consistent answers correctly identified:  {consistent_correct}/{total} ({consistent_correct/total*100:.0f}%)")
print(f"   • Contradictions detected:                  {contradictory_detected}/{total} ({contradictory_detected/total*100:.0f}%)")
print(f"   • Overall accuracy:                         {(consistent_correct+contradictory_detected)/(total*2)*100:.0f}%")


### 3.3 Analysis by Contradiction Category


In [ ]:
# Analyze by category
print("\n" + "="*80)
print("📊 ANALYSIS BY CONTRADICTION CATEGORY")
print("="*80)

categories = results_df["category"].unique()

for cat in categories:
    cat_results = results_df[results_df["category"] == cat]
    consistent_contr = cat_results[cat_results["type"] == "Consistent"]["contradiction_score"].values[0]
    contradictory_contr = cat_results[cat_results["type"] == "Contradictory"]["contradiction_score"].values[0]
    score_gap = contradictory_contr - consistent_contr
    
    print(f"\n📌 {cat}:")
    print(f"   Consistent - Contradiction Score:     {consistent_contr:.3f}")
    print(f"   Contradictory - Contradiction Score:  {contradictory_contr:.3f}")
    print(f"   Detection Gap:                        {score_gap:.3f} {'✓' if score_gap > 0.30 else '⚠️'}")

# Overall statistics
print(f"\n{'='*80}")
print("📊 OVERALL STATISTICS")
print("="*80)

consistent_scores = results_df[results_df["type"] == "Consistent"]["contradiction_score"]
contradictory_scores = results_df[results_df["type"] == "Contradictory"]["contradiction_score"]

print(f"\n📗 CONSISTENT ANSWERS (Contradiction Scores - lower is better):")
print(f"   Mean:  {consistent_scores.mean():.3f}")
print(f"   Min:   {consistent_scores.min():.3f}")
print(f"   Max:   {consistent_scores.max():.3f}")

print(f"\n📕 CONTRADICTORY ANSWERS (Contradiction Scores - higher indicates contradiction):")
print(f"   Mean:  {contradictory_scores.mean():.3f}")
print(f"   Min:   {contradictory_scores.min():.3f}")
print(f"   Max:   {contradictory_scores.max():.3f}")

print(f"\n📉 SEPARATION:")
avg_gap = contradictory_scores.mean() - consistent_scores.mean()
print(f"   Average Gap: {avg_gap:.3f}")
print(f"   Separation Quality: {'Excellent ✓' if avg_gap > 0.40 else 'Good ✓' if avg_gap > 0.25 else 'Moderate ⚠️' if avg_gap > 0.15 else 'Poor ❌'}")


## 4. Interactive Testing: Try Your Own Examples


In [ ]:
def test_contradiction(context: str, answer: str):
    """
    Test if an answer contradicts a given context.
    
    Usage:
        test_contradiction(
            context="The sky is blue during clear days.",
            answer="The sky appears red during clear days."
        )
    """
    result = detector.detect_contradiction(answer, context)
    
    print("="*60)
    print("🔍 CONTRADICTION TEST")
    print("="*60)
    print(f"\n📄 Context:\n{context}")
    print(f"\n💬 Answer:\n{answer}")
    print(f"\n📊 NLI SCORES:")
    print(f"   • Entailment:     {result['nli_entailment']:.3f}")
    print(f"   • Neutral:        {result['nli_neutral']:.3f}")
    print(f"   • Contradiction:  {result['nli_contradiction']:.3f} ← KEY")
    print(f"\n📊 OTHER SCORES:")
    print(f"   • Semantic:       {result['semantic_score']:.3f}")
    print(f"   • Combined:       {result['combined_score']:.3f}")
    print(f"\n🎯 RESULT: {result['category']}")
    print(f"   Risk Level: {result['risk_level']}")
    print("="*60)
    
    return result

# Example: Consistent answer
print("TEST 1: Consistent Answer")
test_contradiction(
    context="The Eiffel Tower is located in Paris, France, and stands 330 meters tall.",
    answer="The Eiffel Tower, at 330 meters, is a famous landmark in Paris, France."
)


In [ ]:
# Example: Contradictory answer
print("TEST 2: Contradictory Answer")
test_contradiction(
    context="The Eiffel Tower is located in Paris, France, and stands 330 meters tall.",
    answer="The Eiffel Tower is in London, England, and is only 100 meters tall."
)


In [ ]:
# Example: Subtle contradiction
print("TEST 3: Subtle Contradiction (Directional)")
test_contradiction(
    context="Company profits increased by 20% in the fourth quarter of 2024.",
    answer="Company profits declined significantly in the fourth quarter of 2024."
)


## 5. Score Comparison Visualization


In [ ]:
# Visual comparison of contradiction scores
consistent_scores = results_df[results_df["type"] == "Consistent"]["contradiction_score"]
contradictory_scores = results_df[results_df["type"] == "Contradictory"]["contradiction_score"]

print("="*60)
print("📊 CONTRADICTION SCORE COMPARISON")
print("="*60)
print("\n(Higher contradiction score = more contradictory)")

print("\n📗 CONSISTENT ANSWERS:")
print(f"   Average:  {np.mean(consistent_scores):.3f}")
print(f"   Min:      {np.min(consistent_scores):.3f}")
print(f"   Max:      {np.max(consistent_scores):.3f}")

print("\n📕 CONTRADICTORY ANSWERS:")
print(f"   Average:  {np.mean(contradictory_scores):.3f}")
print(f"   Min:      {np.min(contradictory_scores):.3f}")
print(f"   Max:      {np.max(contradictory_scores):.3f}")

print("\n📉 SEPARATION:")
score_diff = np.mean(contradictory_scores) - np.mean(consistent_scores)
print(f"   Score Gap: {score_diff:.3f}")

# Visual bar representation (inverted - lower is better for consistent)
print("\n📊 VISUAL COMPARISON (Contradiction Score):")
print(f"   Consistent:    {'░' * int(np.mean(consistent_scores) * 20):<20} {np.mean(consistent_scores):.2f} (low = good)")
print(f"   Contradictory: {'█' * int(np.mean(contradictory_scores) * 20):<20} {np.mean(contradictory_scores):.2f} (high = bad)")
print(f"   Threshold:     {'─' * 8}│{'─' * 11}  0.40 (contradiction threshold)")

# Per-category visualization
print("\n📊 CONTRADICTION SCORES BY CATEGORY:")
for cat in categories:
    cat_data = results_df[results_df["category"] == cat]
    c_score = cat_data[cat_data["type"] == "Consistent"]["contradiction_score"].values[0]
    x_score = cat_data[cat_data["type"] == "Contradictory"]["contradiction_score"].values[0]
    print(f"   {cat:12} Consistent: {c_score:.2f} {'░' * int(c_score * 10):<10}  |  Contradictory: {x_score:.2f} {'█' * int(x_score * 10):<10}")


## 6. Key Takeaways


In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                    KEY TAKEAWAYS: CONTRADICTION DETECTION                    ║
╚══════════════════════════════════════════════════════════════════════════════╝

✅ WHAT WE DEMONSTRATED:

1. CONTRADICTION TYPES DETECTED:
   • Numeric contradictions (wrong numbers, percentages)
   • Logical contradictions (direct negations)
   • Attribute contradictions (wrong properties)
   • Temporal contradictions (wrong dates, sequences)
   • Directional contradictions (increase vs decrease)
   • Outcome contradictions (success vs failure)
   • Identity contradictions (wrong creators, origins)

2. WHY NLI IS IDEAL FOR CONTRADICTION:
   • NLI models are trained specifically for this task
   • Output includes explicit "contradiction" score
   • Can detect subtle logical inconsistencies
   • Works across different domains

3. KEY METRIC: NLI CONTRADICTION SCORE
   • Low score (< 0.2): Answer is consistent with context
   • Medium score (0.2 - 0.4): Minor inconsistency
   • High score (0.4 - 0.7): Likely contradiction
   • Very high (> 0.7): Clear contradiction

4. RECOMMENDED THRESHOLDS:
   • Contradiction < 0.20: Safe - consistent response
   • Contradiction 0.20-0.40: Review - minor issues
   • Contradiction 0.40-0.70: Flag - likely contradiction
   • Contradiction > 0.70: Block - clear contradiction

═══════════════════════════════════════════════════════════════════════════════

🔗 NEXT STEPS:
   • Integrate with MLflow for production monitoring
   • Set up automated alerts for high contradiction scores
   • Combine with fact-checking systems
   • Build domain-specific contradiction rules

═══════════════════════════════════════════════════════════════════════════════

⚠️ IMPORTANT NOTES:
   • NLI models may struggle with very domain-specific content
   • Subtle contradictions may have lower scores
   • Consider context length limitations of NLI models
   • Use ensemble approaches for high-stakes applications

═══════════════════════════════════════════════════════════════════════════════
""")
